In [2]:
print('hello, world!')

hello, world!


because we are using submodules, there can be some issues importing packages from our exising kernel (which is built off the .toml file). so, we manually import them here.

In [3]:
import sys
absolute_path = '/Users/denver/Documents/semiotic/yode/sql/internal/autoSQL/autosql/autosql/eval/test-suite-sql-eval'
sys.path.append(absolute_path)

In [4]:
!pip install nltk
!pip install nest_asyncio


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip


In [5]:
from datasets import Dataset, DatasetDict
import datasets

import os
import json
import asyncio
import nest_asyncio
nest_asyncio.apply()

from process_sql import get_schema, Schema, get_sql
from exec_eval import eval_exec_match
from evaluation import evaluate, build_foreign_key_map_from_json, Evaluator, build_valid_col_units, rebuild_sql_val, rebuild_sql_col, print_scores

In [5]:
ds = datasets.load_dataset('alagaesia/spider_dev')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1034 [00:00<?, ? examples/s]

we want to test that we can run the test suite eval and correctly report values. to do so, we are going to create a prediction value that should be 100% accurate, simply by replicating the gold as the prediction. this should give us a basic workflow for testing the test suite.

In [6]:
ds

DatasetDict({
    train: Dataset({
        features: ['db_id', 'query', 'question', 'create_w_keys', 'create_wo_keys', 'difficulty'],
        num_rows: 1034
    })
})

In [7]:
ds['train'][0]

{'db_id': 'concert_singer',
 'query': 'SELECT count(*) FROM singer',
 'question': 'How many singers do we have?',
 'create_w_keys': 'CREATE TABLE "stadium" ( "Stadium_ID" int, "Location" text, "Name" text, "Capacity" int, "Highest" int, "Lowest" int, "Average" int, PRIMARY KEY ("Stadium_ID") ); CREATE TABLE "singer" ( "Singer_ID" int, "Name" text, "Country" text, "Song_Name" text, "Song_release_year" text, "Age" int, "Is_male" bool, PRIMARY KEY ("Singer_ID") ); CREATE TABLE "concert" ( "concert_ID" int, "concert_Name" text, "Theme" text, "Stadium_ID" text, "Year" text, PRIMARY KEY ("concert_ID"), FOREIGN KEY ("Stadium_ID") REFERENCES "stadium"("Stadium_ID") ); CREATE TABLE "singer_in_concert" ( "concert_ID" int, "Singer_ID" text, PRIMARY KEY ("concert_ID","Singer_ID"), FOREIGN KEY ("concert_ID") REFERENCES "concert"("concert_ID"), FOREIGN KEY ("Singer_ID") REFERENCES "singer"("Singer_ID") ); ',
 'create_wo_keys': 'CREATE TABLE stadium (Stadium_ID INT, Location TEXT, Name TEXT, Capacity

In [ ]:
evaluator = Evaluator()

In [ ]:
db = query = ds['train'][0]['db_id']
db

In [ ]:
db_path = os.path.join('test-suite-sql-eval/database', db, db + ".sqlite")

In [ ]:
schema = Schema(get_schema(db_path))

In [ ]:
query = ds['train'][0]['query']

g_sql = get_sql(schema, query)

evaluator.eval_hardness(g_sql)

In [ ]:
def hardness(
    db_id: str,
    query: str,
    evaluator: Evaluator,
): 
    
    db_path = os.path.join('test-suite-sql-eval/database', db_id, db_id + ".sqlite")
    schema = Schema(get_schema(db_path))

    g_sql = get_sql(schema, query)

    return evaluator.eval_hardness(g_sql)

def map_hardness(
    dataset: Dataset,
    db_id_column: str,
    query_column: str,
    evaluator: Evaluator,
): 
    
    db_id = dataset[db_id_column]
    query = dataset[query_column]

    print(db_id)
    print(query)

    level = hardness(db_id, query, evaluator)

    return {'difficulty': level}

#db = query = ds['train'][0]['db_id']
#query = ds['train'][0]['query']

#hardness(db, query, evaluator)

ds['train'] = ds['train'].map(map_hardness, fn_kwargs = {'db_id_column': 'db_id', 'query_column': 'query', 'evaluator': evaluator}, load_from_cache_file = False)

#ds = map_hardness(ds['train'], 'db_id', 'query', evaluator)

In [ ]:
ds['train'][370]

In [ ]:
ds.push_to_hub('alagaesia/spider_dev')

In [ ]:
ds

In [ ]:
diff = {}

for i in range(len(ds['train'])):

    if ds['train'][i]['difficulty'] in diff:
        diff[ds['train'][i]['difficulty']] += 1
    else:
        diff[ds['train'][i]['difficulty']] = 1

In [ ]:
diff

In [ ]:
with open('gold.txt', 'w') as file:

    for i in range(len(ds['train'])):

        query = ds['train'][i]['query']
        db_id = ds['train'][i]['db_id']

        file.write(f"{query}\t{db_id}\n")

In [ ]:
with open('predict.txt', 'w') as file:

    for i in range(len(ds['train'])):

        query = ds['train'][i]['query']
        #db_id = ds['train'][i]['db_id']

        file.write(f"{query}\n") # \t{db_id}

# clean up the prediction file

In [15]:
data = DatasetDict.load_from_disk('data_7b_predictions')

In [16]:
llama = DatasetDict.load_from_disk('llama_7b')

In [54]:
data = Dataset.load_from_disk('all_predictions')

In [55]:
data

Dataset({
    features: ['db_id', 'query', 'question', 'create_w_keys', 'create_wo_keys', 'difficulty', 'zero_shot_request', 'defog_sqlcoder_7b', 'defog_sqlcoder_7b_latency', 'mistral_7b_instruct', 'mistral_7b_instruct_latency', 'llama_7b', 'llama_7b_latency', 'zero_shot_request_complex', 'mistral_7b_instruct_complex', 'mistral_7b_instruct_complex_latency', 'defog_sqlcoder_7b_complex', 'defog_sqlcoder_7b_complex_latency', 'llama_7b_complex', 'llama_7b_complex_latency', 'gpt_3_5_complex', 'gpt_3_5'],
    num_rows: 209
})

In [56]:
from typing import Optional
import re

def dep_response_parser(
        prediction: str,
    ): 
         
        pattern = r"SELECT.*?(?=\n|\[/|,\[INST\])"
        matches = re.findall(pattern, prediction, re.DOTALL)

        return matches

def response_parser(prediction: str):

    prediction = prediction.replace('\n', ' ')
    prediction = prediction.replace('\t', ' ')

    pattern = r"\bSELECT\b.*?(?=;|\n|#|\[/|,\[INST\]|$)"
    matches = re.findall(pattern, prediction, re.DOTALL | re.IGNORECASE)

    matches = [match.strip() for match in matches if match.strip()]

    if len(matches) == 0:
        return ''
    else:
        return matches[0]

    #return matches[0]

def dataset_response_parser(
        dataset,
        prediction_column: str,
        result_label: Optional[str] = None,
    ): 
        
        prediction = dataset[prediction_column]
         
        #pattern = r"SELECT.*?(?=\n|\[/|,\[INST\])"
        #matches = re.findall(pattern, prediction, re.DOTALL)

        matches = response_parser(prediction)

        if result_label is None: 
            result_label = prediction_column + '_cleaned'

        return {result_label: matches}

In [57]:
data = data.map(dataset_response_parser, fn_kwargs = {'prediction_column': 'llama_7b_complex'}, load_from_cache_file = False)

Map:   0%|          | 0/209 [00:00<?, ? examples/s]

In [58]:
data = data.map(dataset_response_parser, fn_kwargs = {'prediction_column': 'llama_7b'}, load_from_cache_file = False)

Map:   0%|          | 0/209 [00:00<?, ? examples/s]

In [59]:
data = data.map(dataset_response_parser, fn_kwargs = {'prediction_column': 'defog_sqlcoder_7b_complex'}, load_from_cache_file = False)

Map:   0%|          | 0/209 [00:00<?, ? examples/s]

In [60]:
data = data.map(dataset_response_parser, fn_kwargs = {'prediction_column': 'defog_sqlcoder_7b'}, load_from_cache_file = False)

Map:   0%|          | 0/209 [00:00<?, ? examples/s]

In [61]:
data = data.map(dataset_response_parser, fn_kwargs = {'prediction_column': 'mistral_7b_instruct_complex'}, load_from_cache_file = False)

Map:   0%|          | 0/209 [00:00<?, ? examples/s]

In [62]:
data = data.map(dataset_response_parser, fn_kwargs = {'prediction_column': 'mistral_7b_instruct'}, load_from_cache_file = False)

Map:   0%|          | 0/209 [00:00<?, ? examples/s]

In [65]:
data.save_to_disk('all_predictions_cleaned')

Saving the dataset (0/1 shards):   0%|          | 0/209 [00:00<?, ? examples/s]

# 

# Evaluate with the test suite

In [67]:
with open('gold.txt', 'w') as file:

    for i in range(len(data)):

        query = data[i]['query']
        db_id = data[i]['db_id']

        file.write(f"{query}\t{db_id}\n")

In [78]:
with open('llama_7b_complex_predict.txt', 'w') as file:

    for i in range(len(data)):

        query = data[i]['llama_7b_complex_cleaned']
        
        if query == '':
            query = 'SELECT'

        file.write(f"{query}\n") # \t{db_id}

In [80]:


with open('llama_7b_predict.txt', 'w') as file:

    for i in range(len(data)):

        query = data[i]['llama_7b_cleaned']

        if query == '':
            query = 'SELECT'

        file.write(f"{query}\n") # \t{db_id}

with open('defog_sqlcoder_7b_complex_predict.txt', 'w') as file:

    for i in range(len(data)):

        query = data[i]['defog_sqlcoder_7b_complex_cleaned']

        if query == '':
            query = 'SELECT'

        file.write(f"{query}\n") # \t{db_id}

with open('defog_sqlcoder_7b_predict.txt', 'w') as file:

    for i in range(len(data)):

        query = data[i]['defog_sqlcoder_7b_cleaned']

        if query == '':
            query = 'SELECT'

        file.write(f"{query}\n") # \t{db_id}

with open('mistral_7b_instruct_complex_predict.txt', 'w') as file:

    for i in range(len(data)):

        query = data[i]['mistral_7b_instruct_complex_cleaned']

        if query == '':
            query = 'SELECT'

        file.write(f"{query}\n") # \t{db_id}

with open('mistral_7b_instruct_predict.txt', 'w') as file:

    for i in range(len(data)):

        query = data[i]['mistral_7b_instruct_cleaned']

        if query == '':
            query = 'SELECT'

        file.write(f"{query}\n") # \t{db_id}

In [73]:
data[110]

{'db_id': 'student_transcripts_tracking',
 'query': 'SELECT line_1 ,  line_2 FROM addresses',
 'question': 'what are all the addresses including line 1 and line 2?',
 'create_w_keys': 'CREATE TABLE `Addresses` ( `address_id` INTEGER PRIMARY KEY, `line_1` VARCHAR(255), `line_2` VARCHAR(255), `line_3` VARCHAR(255), `city` VARCHAR(255), `zip_postcode` VARCHAR(20), `state_province_county` VARCHAR(255), `country` VARCHAR(255), `other_address_details` VARCHAR(255) ); CREATE TABLE `Courses` ( `course_id` INTEGER PRIMARY KEY, `course_name` VARCHAR(255), `course_description` VARCHAR(255), `other_details` VARCHAR(255) ); CREATE TABLE `Departments` ( `department_id` INTEGER PRIMARY KEY, `department_name` VARCHAR(255), `department_description` VARCHAR(255), `other_details` VARCHAR(255) ); CREATE TABLE `Degree_Programs` ( `degree_program_id` INTEGER PRIMARY KEY, `department_id` INTEGER NOT NULL, `degree_summary_name` VARCHAR(255), `degree_summary_description` VARCHAR(255), `other_details` VARCHAR(2

In [90]:
# evaluate(args.gold, args.pred, args.db, args.etype, kmaps, args.plug_value, args.keep_distinct, args.progress_bar_for_each_datapoint)
# python3 evaluation.py --gold evaluation_examples/gold.txt --pred evaluation_examples/predict.txt --etype all --db database/ --table tables.json --plug_value --keep_distinct --progress_bar_for_each_datapoint

gold_path = "gold.txt"
pred_path = "llama_7b_complex_predict.txt"
db_dir = "draft/database/"
table_path = "draft/tables.json"
etype = "all"
plug_value = False
keep_distinct = True
progress_bar_for_each_datapoint = False

llama_7b_pred_path = "llama_7b_predict.txt"
llama_7b_complex_pred_path = "llama_7b_complex_predict.txt"
defog_sqlcoder_7b_pred_path = "defog_sqlcoder_7b_predict.txt"
defog_sqlcoder_7b_complex_pred_path = "defog_sqlcoder_7b_complex_predict.txt"
mistral_7b_instruct_pred_path = "mistral_7b_instruct_predict.txt"
mistral_7b_instruct_complex_pred_path = "mistral_7b_instruct_complex_predict.txt"

if etype in ['all', 'match']:
    kmaps = build_foreign_key_map_from_json(table_path)
else:
    kmaps = None

In [91]:
llama_7b_scores = await evaluate(gold_path, llama_7b_pred_path, db_dir, etype, kmaps, plug_value, keep_distinct, progress_bar_for_each_datapoint)

easy pred: SELECT
easy gold: SELECT count(*) FROM singer

medium pred: SELECT
medium gold: SELECT name ,  country ,  age FROM singer ORDER BY age DESC

medium pred: SELECT
medium gold: SELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  'France'

medium pred: SELECT
medium gold: SELECT song_name ,  song_release_year FROM singer ORDER BY age LIMIT 1

easy pred: SELECT
easy gold: SELECT DISTINCT country FROM singer WHERE age  >  20

hard pred: SELECT
hard gold: SELECT song_name FROM singer WHERE age  >  (SELECT avg(age) FROM singer)

extra pred: SELECT
extra gold: SELECT T2.name ,  T2.capacity FROM concert AS T1 JOIN stadium AS T2 ON T1.stadium_id  =  T2.stadium_id WHERE T1.year  >=  2014 GROUP BY T2.stadium_id ORDER BY count(*) DESC LIMIT 1

extra pred: SELECT
extra gold: select t2.name ,  t2.capacity from concert as t1 join stadium as t2 on t1.stadium_id  =  t2.stadium_id where t1.year  >  2013 group by t2.stadium_id order by count(*) desc limit 1

hard pred: SELECT
h

In [92]:
llama_7b_complex_scores = await evaluate(gold_path, llama_7b_complex_pred_path, db_dir, etype, kmaps, plug_value, keep_distinct, progress_bar_for_each_datapoint)

medium pred: SELECT name, country, age FROM singer WHERE age = (SELECT MIN(age)
medium gold: SELECT name ,  country ,  age FROM singer ORDER BY age DESC

medium pred: SELECT average(age), MIN(age), MAX(age) FROM singer WHERE country =
medium gold: SELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  'France'

medium pred: SELECT Name, Song_release_year FROM singer WHERE Age < (SELECT MIN(Age
medium gold: SELECT song_name ,  song_release_year FROM singer ORDER BY age LIMIT 1

easy pred: SELECT "Country" FROM "singer" WHERE "Age" > 20
easy gold: SELECT DISTINCT country FROM singer WHERE age  >  20

hard pred: SELECT "Name" FROM "singer_in_concert" WHERE "Singer
hard gold: SELECT song_name FROM singer WHERE age  >  (SELECT avg(age) FROM singer)

extra pred: SELECT name, capacity FROM stadium WHERE year > 2014
extra gold: SELECT T2.name ,  T2.capacity FROM concert AS T1 JOIN stadium AS T2 ON T1.stadium_id  =  T2.stadium_id WHERE T1.year  >=  2014 GROUP BY T2.stadium_id ORD

In [93]:
defog_sqlcoder_7b_scores = await evaluate(gold_path, defog_sqlcoder_7b_pred_path, db_dir, etype, kmaps, plug_value, keep_distinct, progress_bar_for_each_datapoint)

medium pred: SELECT singer.name, singer.country, singer.age FROM singer ORDER BY singer
medium gold: SELECT name ,  country ,  age FROM singer ORDER BY age DESC

medium pred: SELECT AVG(singer.age) AS average_age, MIN(singer.
medium gold: SELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  'France'

medium pred: SELECT s.name, to_number(s.song_release_year,
medium gold: SELECT song_name ,  song_release_year FROM singer ORDER BY age LIMIT 1

easy pred: SELECT s.country FROM singer s JOIN singer_in_concert sic ON s
easy gold: SELECT DISTINCT country FROM singer WHERE age  >  20

hard pred: SELECT s.song_name FROM singer s JOIN singer_in_concert
hard gold: SELECT song_name FROM singer WHERE age  >  (SELECT avg(age) FROM singer)

extra pred: SELECT s.name, s.capacity FROM stadium s JOIN concert c ON s.
extra gold: SELECT T2.name ,  T2.capacity FROM concert AS T1 JOIN stadium AS T2 ON T1.stadium_id  =  T2.stadium_id WHERE T1.year  >=  2014 GROUP BY T2.stadium_id ORDER BY c

In [94]:
defog_sqlcoder_7b_complex_scores = await evaluate(gold_path, defog_sqlcoder_7b_complex_pred_path, db_dir, etype, kmaps, plug_value, keep_distinct, progress_bar_for_each_datapoint)

medium pred: SELECT singer.name, singer.country, singer.age FROM singer ORDER BY singer.
medium gold: SELECT name ,  country ,  age FROM singer ORDER BY age DESC

medium pred: SELECT AVG(singer.age), MIN(singer.age), MAX(s
medium gold: SELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  'France'

medium pred: SELECT s.name, to_number(s.song_release_year, '
medium gold: SELECT song_name ,  song_release_year FROM singer ORDER BY age LIMIT 1

easy pred: SELECT s.name, to_number(s.age,'9999') AS
easy gold: SELECT DISTINCT country FROM singer WHERE age  >  20

hard pred: SELECT s.name, to_number(s.song_release_year, '
hard gold: SELECT song_name FROM singer WHERE age  >  (SELECT avg(age) FROM singer)

extra pred: SELECT s.name, s.capacity FROM stadium s JOIN concert c ON s.stad
extra gold: SELECT T2.name ,  T2.capacity FROM concert AS T1 JOIN stadium AS T2 ON T1.stadium_id  =  T2.stadium_id WHERE T1.year  >=  2014 GROUP BY T2.stadium_id ORDER BY count(*) DESC LIMIT 1

extr

In [95]:
mistral_7b_instruct_scores = await evaluate(gold_path, mistral_7b_instruct_pred_path, db_dir, etype, kmaps, plug_value, keep_distinct, progress_bar_for_each_datapoint)

medium pred: SELECT Singer_Name, Country, Age FROM singer ORDER BY Age DESC
medium gold: SELECT name ,  country ,  age FROM singer ORDER BY age DESC

medium pred: SELECT AVG(Age), MIN(Age), MAX(Age) FROM
medium gold: SELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  'France'

medium pred: SELECT "Singer_Name", "Song_release_year" FROM "s
medium gold: SELECT song_name ,  song_release_year FROM singer ORDER BY age LIMIT 1

hard pred: SELECT "Song_Name" FROM "singer"  INNER JOIN
hard gold: SELECT song_name FROM singer WHERE age  >  (SELECT avg(age) FROM singer)

extra pred: SELECT Stadium_Name, Stadium_Capacity FROM stadium  JOIN concert ON stadium.
extra gold: SELECT T2.name ,  T2.capacity FROM concert AS T1 JOIN stadium AS T2 ON T1.stadium_id  =  T2.stadium_id WHERE T1.year  >=  2014 GROUP BY T2.stadium_id ORDER BY count(*) DESC LIMIT 1

extra pred: SELECT Stadium_Name, Stadium_Capacity FROM stadium  JOIN concert ON stadium.
extra gold: select t2.name ,  t2.capacity 

In [96]:
mistral_7b_instruct_scores = await evaluate(gold_path, mistral_7b_instruct_pred_path, db_dir, etype, kmaps, plug_value, keep_distinct, progress_bar_for_each_datapoint)

medium pred: SELECT Singer_Name, Country, Age FROM singer ORDER BY Age DESC
medium gold: SELECT name ,  country ,  age FROM singer ORDER BY age DESC

medium pred: SELECT AVG(Age), MIN(Age), MAX(Age) FROM
medium gold: SELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  'France'

medium pred: SELECT "Singer_Name", "Song_release_year" FROM "s
medium gold: SELECT song_name ,  song_release_year FROM singer ORDER BY age LIMIT 1

hard pred: SELECT "Song_Name" FROM "singer"  INNER JOIN
hard gold: SELECT song_name FROM singer WHERE age  >  (SELECT avg(age) FROM singer)

extra pred: SELECT Stadium_Name, Stadium_Capacity FROM stadium  JOIN concert ON stadium.
extra gold: SELECT T2.name ,  T2.capacity FROM concert AS T1 JOIN stadium AS T2 ON T1.stadium_id  =  T2.stadium_id WHERE T1.year  >=  2014 GROUP BY T2.stadium_id ORDER BY count(*) DESC LIMIT 1

extra pred: SELECT Stadium_Name, Stadium_Capacity FROM stadium  JOIN concert ON stadium.
extra gold: select t2.name ,  t2.capacity 

In [97]:
task = await evaluate(gold_path, pred_path, db_dir, etype, kmaps, plug_value, keep_distinct, progress_bar_for_each_datapoint)

medium pred: SELECT name, country, age FROM singer WHERE age = (SELECT MIN(age)
medium gold: SELECT name ,  country ,  age FROM singer ORDER BY age DESC

medium pred: SELECT average(age), MIN(age), MAX(age) FROM singer WHERE country =
medium gold: SELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  'France'

medium pred: SELECT Name, Song_release_year FROM singer WHERE Age < (SELECT MIN(Age
medium gold: SELECT song_name ,  song_release_year FROM singer ORDER BY age LIMIT 1

easy pred: SELECT "Country" FROM "singer" WHERE "Age" > 20
easy gold: SELECT DISTINCT country FROM singer WHERE age  >  20

hard pred: SELECT "Name" FROM "singer_in_concert" WHERE "Singer
hard gold: SELECT song_name FROM singer WHERE age  >  (SELECT avg(age) FROM singer)

extra pred: SELECT name, capacity FROM stadium WHERE year > 2014
extra gold: SELECT T2.name ,  T2.capacity FROM concert AS T1 JOIN stadium AS T2 ON T1.stadium_id  =  T2.stadium_id WHERE T1.year  >=  2014 GROUP BY T2.stadium_id ORD

In [100]:
import pickle

In [101]:
scores_7b = {
    'llama_7b_scores': llama_7b_scores,
    'llama_7b_complex_scores': llama_7b_complex_scores,
    'defog_sqlcoder_7b_scores': defog_sqlcoder_7b_scores,
    'defog_sqlcoder_7b_complex_scores': defog_sqlcoder_7b_complex_scores,
    'mistral_7b_instruct_scores': mistral_7b_instruct_scores,
    'mistral_7b_instruct_scores': mistral_7b_instruct_scores,
}

In [102]:
# pickle scores_7b
with open('scores_7b.pickle', 'wb') as handle:
    pickle.dump(scores_7b, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [103]:
import pandas as pd

def flatten_scores(scores):
    flat_scores = {}
    for difficulty_level, metrics in scores.items():
        for metric, values in metrics.items():
            if isinstance(values, dict):
                for sub_metric, value in values.items():
                    flat_key = f"{difficulty_level}_{metric}_{sub_metric}"
                    flat_scores[flat_key] = value
            else:
                flat_key = f"{difficulty_level}_{metric}"
                flat_scores[flat_key] = values
    return flat_scores

flattened_data = {}
for model, model_scores in scores_7b.items():
    flattened_data[model] = flatten_scores(model_scores)

df = pd.DataFrame(flattened_data).T
print(df)

                                  easy_count  easy_execution_accuracy  \
llama_7b_scores                         56.0                 0.000000   
llama_7b_complex_scores                 56.0                 0.339286   
defog_sqlcoder_7b_scores                56.0                 0.160714   
defog_sqlcoder_7b_complex_scores        56.0                 0.464286   
mistral_7b_instruct_scores              56.0                 0.642857   

                                  easy_exact_matching_accuracy  \
llama_7b_scores                                       0.000000   
llama_7b_complex_scores                               0.303571   
defog_sqlcoder_7b_scores                              0.089286   
defog_sqlcoder_7b_complex_scores                      0.285714   
mistral_7b_instruct_scores                            0.607143   

                                  easy_partial_matching_accuracy_select  \
llama_7b_scores                                                0.000000   
llama_7b_compl

In [98]:
mistral_7b_instruct_scores

{'easy': {'count': 56,
  'execution_accuracy': 0.6428571428571429,
  'exact_matching_accuracy': 0.6071428571428571,
  'partial_matching_accuracy': {'select': 0.8717948717948718,
   'select(no AGG)': 0.8974358974358975,
   'where': 1.0,
   'where(no OP)': 1.0,
   'group(no Having)': 0,
   'group': 0,
   'order': 1.0,
   'and/or': 1.0,
   'IUEN': 0,
   'keywords': 1.0},
  'partial_matching_recall': {'select': 0.6071428571428571,
   'select(no AGG)': 0.625,
   'where': 0.4090909090909091,
   'where(no OP)': 0.4090909090909091,
   'group(no Having)': 0.0,
   'group': 0.0,
   'order': 1.0,
   'and/or': 1.0,
   'IUEN': 0,
   'keywords': 0.5172413793103449},
  'partial_matching_f1': {'select': 0.7157894736842105,
   'select(no AGG)': 0.736842105263158,
   'where': 0.5806451612903226,
   'where(no OP)': 0.5806451612903226,
   'group(no Having)': 1,
   'group': 1,
   'order': 1.0,
   'and/or': 1.0,
   'IUEN': 1,
   'keywords': 0.6818181818181819}},
 'medium': {'count': 59,
  'execution_accuracy

In [99]:
def display_metrics(metrics):
    for difficulty, metrics_data in metrics.items():
        print(f"----- {difficulty.upper()} -----")
        print(f"Count: {metrics_data['count']}")
        print(f"Execution Accuracy: {metrics_data['execution_accuracy']:.2f}")
        print(f"Exact Matching Accuracy: {metrics_data['exact_matching_accuracy']:.2f}")
        
        for metric_type in ["partial_matching_accuracy", "partial_matching_recall", "partial_matching_f1"]:
            print(f"\n{metric_type.replace('_', ' ').capitalize()}:")
            for key, value in metrics_data[metric_type].items():
                print(f"  {key}: {value:.2f}")
        
        print("\n" + "-"*40)

display_metrics(mistral_7b_instruct_scores)


----- EASY -----
Count: 56
Execution Accuracy: 0.64
Exact Matching Accuracy: 0.61

Partial matching accuracy:
  select: 0.87
  select(no AGG): 0.90
  where: 1.00
  where(no OP): 1.00
  group(no Having): 0.00
  group: 0.00
  order: 1.00
  and/or: 1.00
  IUEN: 0.00
  keywords: 1.00

Partial matching recall:
  select: 0.61
  select(no AGG): 0.62
  where: 0.41
  where(no OP): 0.41
  group(no Having): 0.00
  group: 0.00
  order: 1.00
  and/or: 1.00
  IUEN: 0.00
  keywords: 0.52

Partial matching f1:
  select: 0.72
  select(no AGG): 0.74
  where: 0.58
  where(no OP): 0.58
  group(no Having): 1.00
  group: 1.00
  order: 1.00
  and/or: 1.00
  IUEN: 1.00
  keywords: 0.68

----------------------------------------
----- MEDIUM -----
Count: 59
Execution Accuracy: 0.24
Exact Matching Accuracy: 0.22

Partial matching accuracy:
  select: 0.95
  select(no AGG): 0.95
  where: 1.00
  where(no OP): 1.00
  group(no Having): 1.00
  group: 1.00
  order: 0.40
  and/or: 0.93
  IUEN: 0.00
  keywords: 0.67

Par

# PyTorch Metrics